In [1]:
import pandas as pd 
import datetime
from shapely.geometry import Point, shape, Polygon
import geopandas as gpd
import shapely.speedups
from pprint import pprint 

# US-GS dataset

We use the [US-GS dataset](https://www.usgs.gov/programs/earthquake-hazards/earthquakes) to acquire overall information about earthquakes. In order to determine a country for this dataset, which only contains coordinates of events, we merge it to the [Natural Earth dataset](https://www.naturalearthdata.com/downloads/10m-cultural-vectors/) which contains country codes and polygons for each country. The data only contains earthquakes registered past 1959, in order to match the dates of the other datasets used in this project.

In [2]:
# Dataset with countries geometry and code

countries_df = gpd.read_file("ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp")

In [3]:
countries_df=countries_df.rename(columns={'SOV_A3':'country code'})

In [4]:
countries_df.head()

,featurecla,scalerank,LABELRANK,SOVEREIGNT,country code,ADM0_DIF,LEVEL,TYPE,ADMIN,ADM0_A3,...,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,geometry
0,Admin-0 country,0,2,Indonesia,IDN,0,2,Sovereign country,Indonesia,IDN,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4..."
1,Admin-0 country,0,3,Malaysia,MYS,0,2,Sovereign country,Malaysia,MYS,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4..."
2,Admin-0 country,0,2,Chile,CHL,0,2,Sovereign country,Chile,CHL,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611..."
3,Admin-0 country,0,3,Bolivia,BOL,0,2,Sovereign country,Bolivia,BOL,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5..."
4,Admin-0 country,0,2,Peru,PER,0,2,Sovereign country,Peru,PER,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832..."


In [5]:
countries_df=countries_df.replace({'country code':'CH1'}, 'CHN')

In [6]:
countries_df=countries_df.replace({'country code':'US1'}, 'USA')
countries_df=countries_df.replace({'country code':'GB1'}, 'GBR')
countries_df=countries_df.replace({'country code':'CU1'}, 'CUB')
countries_df=countries_df.replace({'country code':'FR1'}, 'FRA')
countries_df=countries_df.replace({'country code':'KA1'}, 'KAZ')
countries_df=countries_df.replace({'country code':'AU1'}, 'AUS')
countries_df=countries_df.replace({'country code':'NZ1'}, 'NZL')

In [7]:
countries_df[['country code','SOVEREIGNT']].values

array([['IDN', 'Indonesia'],
       ['MYS', 'Malaysia'],
       ['CHL', 'Chile'],
       ['BOL', 'Bolivia'],
       ['PER', 'Peru'],
       ['ARG', 'Argentina'],
       ['GBR', 'United Kingdom'],
       ['CYP', 'Cyprus'],
       ['IND', 'India'],
       ['CHN', 'China'],
       ['IS1', 'Israel'],
       ['IS1', 'Israel'],
       ['LBN', 'Lebanon'],
       ['ETH', 'Ethiopia'],
       ['SDS', 'South Sudan'],
       ['SOM', 'Somalia'],
       ['KEN', 'Kenya'],
       ['MWI', 'Malawi'],
       ['TZA', 'United Republic of Tanzania'],
       ['SYR', 'Syria'],
       ['SOL', 'Somaliland'],
       ['FRA', 'France'],
       ['SUR', 'Suriname'],
       ['GUY', 'Guyana'],
       ['KOR', 'South Korea'],
       ['PRK', 'North Korea'],
       ['MAR', 'Morocco'],
       ['SAH', 'Western Sahara'],
       ['CRI', 'Costa Rica'],
       ['NIC', 'Nicaragua'],
       ['COG', 'Republic of the Congo'],
       ['COD', 'Democratic Republic of the Congo'],
       ['BTN', 'Bhutan'],
       ['UKR', 'Ukraine'],
  

In [8]:
df_countries_usgs=pd.read_csv('merged_countries.csv',sep=',')

In [9]:

df_geo_usgs= gpd.GeoDataFrame(df_countries_usgs, geometry=gpd.points_from_xy(df_countries_usgs.longitude, df_countries_usgs.latitude))

In [10]:
df_geo_usgs.head()

,Unnamed: 0,geometry,souvereign,admin,index_right,time,latitude,longitude,depth,mag,id,place,Date
0,0,POINT (-74.46600 -8.49800),Peru,Peru,65172,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
1,13753,POINT (-74.46600 -8.49800),Peru,Peru,65172,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
2,0,POINT (-74.46600 -8.49800),Peru,Peru,1,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
3,13753,POINT (-74.46600 -8.49800),Peru,Peru,1,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
4,1,POINT (110.73600 35.51500),China,China,12,2010-01-24 00:00:00+00:00,35.515,110.736,28.7,5.0,usp000h6j0,"28 km ENE of Hancheng, China",2010-01-24


In [11]:
df_geo_usgs=df_geo_usgs.drop(columns=['Unnamed: 0', 'index_right'])

drop_countries = list(set(countries_df.columns)-set(['id','SOVEREIGNT','country code', 'geometry']))

countries_df=countries_df.drop(columns=drop_countries)


In [12]:
#df_country_codes = gpd.sjoin(usgs_country_code, df_geo_usgs, how="inner")
df_usgs_codes = gpd.sjoin(df_geo_usgs, countries_df, how="inner")

<ipython-input-12-97638a1b7b3d>:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  df_usgs_codes = gpd.sjoin(df_geo_usgs, countries_df, how="inner")


In [13]:
df_usgs_codes.head()

,geometry,souvereign,admin,time,latitude,longitude,depth,mag,id,place,Date,index_right,SOVEREIGNT,country code
0,POINT (-74.46600 -8.49800),Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25,4,Peru,PER
1,POINT (-74.46600 -8.49800),Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25,4,Peru,PER
2,POINT (-74.46600 -8.49800),Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25,4,Peru,PER
3,POINT (-74.46600 -8.49800),Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25,4,Peru,PER
97,POINT (-77.71800 -8.80200),Peru,Peru,2010-01-03 00:00:00+00:00,-8.802,-77.718,116.8,5.7,usp000h5na,"16 km E of Huallanca, Peru",2010-01-03,4,Peru,PER


In [14]:
df_usgs_codes =df_usgs_codes.rename(columns={'SOVEREIGNT':'country', 'Date':'date'})

df_usgs_codes.head()

,geometry,souvereign,admin,time,latitude,longitude,depth,mag,id,place,date,index_right,country,country code
0,POINT (-74.46600 -8.49800),Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25,4,Peru,PER
1,POINT (-74.46600 -8.49800),Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25,4,Peru,PER
2,POINT (-74.46600 -8.49800),Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25,4,Peru,PER
3,POINT (-74.46600 -8.49800),Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25,4,Peru,PER
97,POINT (-77.71800 -8.80200),Peru,Peru,2010-01-03 00:00:00+00:00,-8.802,-77.718,116.8,5.7,usp000h5na,"16 km E of Huallanca, Peru",2010-01-03,4,Peru,PER


Finally, we still merge this dataset to the [USGS Earth and Monitoring dataset](https://www.usgs.gov/programs/earthquake-hazards/data) which contains another relevant feature for our models. 

In [15]:
shaking_data = pd.read_csv('usgs_shaking.csv')

In [16]:
shaking_data.head()

,Unnamed: 0,id,place,time,mag,mmi,geometry
0,0,usp00043zh,"20 km SE of Atiquipa, Peru",631703203440,5.5,4.519,POINT Z (-74.245 -15.947 48)
1,1,usp0004417,"86 km NNW of Mangan, India",631852166690,5.5,4.066,POINT Z (88.163 28.225 79.09999999999999)
2,2,usp0004421,"120 km E of W?ngjing, India",631911089210,6.3,4.782,POINT Z (95.241 24.753 119.2)
3,3,usp000443d,"25 km ENE of Kupang, Indonesia",631987906730,5.5,5.051,POINT Z (123.819 -10.073 32.7)
4,4,usp0004453,near the coast of Nicaragua,632114699080,5.6,4.146,POINT Z (-87.489 12.672 86.2)


In [17]:
print(len(shaking_data))
shaking_data['mmi'].isna().sum()

27829


13813

In [18]:
#merge to the usgs dataset
df_usgs_clean= pd.merge(df_usgs_codes, shaking_data[['id', 'mmi']], how= 'left', on='id')
df_usgs_clean.head(50)

,geometry,souvereign,admin,time,latitude,longitude,depth,mag,id,place,date,index_right,country,country code,mmi
0,POINT (-74.46600 -8.49800),Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25,4,Peru,PER,NaN
1,POINT (-74.46600 -8.49800),Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25,4,Peru,PER,NaN
2,POINT (-74.46600 -8.49800),Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25,4,Peru,PER,NaN
3,POINT (-74.46600 -8.49800),Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25,4,Peru,PER,NaN
4,POINT (-77.71800 -8.80200),Peru,Peru,2010-01-03 00:00:00+00:00,-8.802,-77.718,116.8,5.7,usp000h5na,"16 km E of Huallanca, Peru",2010-01-03,4,Peru,PER,NaN
5,POINT (-73.97800 -15.76600),Peru,Peru,2009-12-24 00:00:00+00:00,-15.766,-73.978,59.1,5.6,usp000h5aq,"4 km NNW of Achanizo, Peru",2009-12-24,4,Peru,PER,4.461
6,POINT (-80.21000 -4.60200),Peru,Peru,2009-10-15 00:00:00+00:00,-4.602,-80.210,62.2,5.0,usp000h2z1,"6 km NE of Las Lomas, Peru",2009-10-15,4,Peru,PER,NaN
7,POINT (-70.24800 -15.12100),Peru,Peru,2009-09-05 00:00:00+00:00,-15.121,-70.248,210.2,5.8,usp000h18a,"12 km N of Nicasio, Peru",2009-09-05,4,Peru,PER,NaN
8,POINT (-75.48900 -14.30100),Peru,Peru,2009-08-03 00:00:00+00:00,-14.301,-75.489,45.4,5.3,usp000gzyk,"27 km ESE of Santiago, Peru",2009-08-03,4,Peru,PER,NaN
9,POINT (-75.40600 -12.09700),Peru,Peru,2009-08-01 00:00:00+00:00,-12.097,-75.406,101.3,5.2,usp000gzwe,"8 km SSW of Chambara, Peru",2009-08-01,4,Peru,PER,NaN


In [19]:
#drop columns
df_usgs_clean=df_usgs_clean.drop(columns=['index_right','geometry', 'souvereign', 'admin', 'latitude', 'longitude', 'id', 'place', 'time'])

#drop earthquake duplicates US-GS dataset
print(len(df_usgs_clean))
df_usgs_clean.drop_duplicates(inplace=True)
print(len(df_usgs_clean))

#missing values for mmi
print('missing mmi values:', 100*(df_usgs_clean['mmi'].isna().sum()/len(df_usgs_clean)))

df_usgs_clean.head()

17090
16777
missing mmi values: 67.09185194015616


,depth,mag,date,country,country code,mmi
0,146.7,5.9,2010-01-25,Peru,PER,NaN
4,116.8,5.7,2010-01-03,Peru,PER,NaN
5,59.1,5.6,2009-12-24,Peru,PER,4.461
6,62.2,5.0,2009-10-15,Peru,PER,NaN
7,210.2,5.8,2009-09-05,Peru,PER,NaN


In [20]:
# save usgs dataset to csv

df_usgs_clean.to_csv(r'df_usgs_clean.csv')

# EMDAT Dataset

Now we merge the data to the [EMDAT dataset](https://www.emdat.be/) for international disasters which was already cleaned to only have Earthquake data. 


In [21]:
df_emdat=pd.read_csv('emdat_date.csv',sep=',')
df_emdat.head(100)

,Unnamed: 0,Dis No,Year,Month,Day,Disaster Type,Disaster Subtype,Country,Country Code,Region,...,No Homeless,Total Affected,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,myDt
0,63,1960-0013-CHL,1960,5.0,22.0,Earthquake,Tsunami,Chile,CHL,South America,...,NaN,2003000.0,NaN,NaN,NaN,NaN,550000.0,4813056.0,11.427251,1960-05-22
1,64,1960-0033-DZA,1960,2.0,21.0,Earthquake,Ground movement,Algeria,DZA,Northern Africa,...,1250.0,1250.0,NaN,NaN,NaN,NaN,NaN,NaN,11.427251,1960-02-21
2,65,1961-0016-ETH,1961,6.0,2.0,Earthquake,Ground movement,Ethiopia,ETH,Eastern Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.549601,1961-06-02
3,66,1962-0044-ALB,1962,3.0,18.0,Earthquake,Ground movement,Albania,ALB,Southern Europe,...,NaN,154.0,NaN,NaN,NaN,NaN,NaN,NaN,11.688060,1962-03-18
4,67,1962-0030-COL,1962,7.0,30.0,Earthquake,Ground movement,Colombia,COL,South America,...,NaN,300.0,NaN,NaN,NaN,NaN,NaN,NaN,11.688060,1962-07-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,265,1971-0034-TUR,1971,5.0,22.0,Earthquake,Ground movement,Turkey,TUR,Western Asia,...,27465.0,88665.0,NaN,NaN,NaN,NaN,5000.0,31959.0,15.645257,1971-05-22
96,266,1971-0016-USA,1971,2.0,9.0,Earthquake,Ground movement,United States of America (the),USA,Northern America,...,NaN,2000.0,NaN,NaN,35000.0,223710.0,553000.0,3534617.0,15.645257,1971-02-09
97,267,1972-0027-IRN,1972,4.0,10.0,Earthquake,Ground movement,Iran (Islamic Republic of),IRN,Southern Asia,...,NaN,23458.0,NaN,NaN,NaN,NaN,1000.0,6189.0,16.157212,1972-04-10
98,268,1972-0084-ITA,1972,2.0,4.0,Earthquake,Ground movement,Italy,ITA,Southern Europe,...,NaN,450.0,NaN,NaN,NaN,NaN,NaN,NaN,16.157212,1972-02-04


In [25]:
df_emdat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 35 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Unnamed: 0                                 1235 non-null   int64  
 1   Dis No                                     1235 non-null   object 
 2   Year                                       1235 non-null   int64  
 3   Month                                      1235 non-null   float64
 4   Day                                        1235 non-null   float64
 5   Disaster Type                              1235 non-null   object 
 6   Disaster Subtype                           1233 non-null   object 
 7   Country                                    1235 non-null   object 
 8   Country Code                               1235 non-null   object 
 9   Region                                     1235 non-null   object 
 10  Continent               

In [26]:
# copy dataset for preprocessing

df_emdat_processing = df_emdat.copy()

In [27]:
df_emdat_processing.columns

Index(['Unnamed: 0', 'Dis No', 'Year', 'Month', 'Day', 'Disaster Type',
       'Disaster Subtype', 'Country', 'Country Code', 'Region', 'Continent',
       'Location', 'Associated Dis', 'Associated Dis2', 'OFDA Response',
       'Appeal', 'Declaration', 'Aid Contribution', 'Dis Mag Value',
       'Latitude', 'Longitude', 'Local Time', 'Total Deaths', 'No Injured',
       'No Affected', 'No Homeless', 'Total Affected',
       'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damages ('000 US$)', 'Insured Damages, Adjusted ('000 US$)',
       'Total Damages ('000 US$)', 'Total Damages, Adjusted ('000 US$)', 'CPI',
       'myDt'],
      dtype='object')

In [28]:
drop_emdat = list(set(df_emdat_processing.columns)-
                  set(['Year','Country', 'Country Code', 'Continent', 
                       'Total Affected','CPI','myDt']))

df_emdat_processing = df_emdat_processing.drop(columns=drop_emdat)
df_emdat_processing.head()

,Year,Country,Country Code,Continent,Total Affected,CPI,myDt
0,1960,Chile,CHL,Americas,2003000.0,11.427251,1960-05-22
1,1960,Algeria,DZA,Africa,1250.0,11.427251,1960-02-21
2,1961,Ethiopia,ETH,Africa,NaN,11.549601,1961-06-02
3,1962,Albania,ALB,Europe,154.0,11.688060,1962-03-18
4,1962,Colombia,COL,Americas,300.0,11.688060,1962-07-30


In [29]:
# Dropping missing vaues for Total affected

print(len(df_emdat_processing))
df_emdat_processing.dropna(subset=['Total Affected'], inplace=True)
print(len(df_emdat_processing))

1235
1136


In [30]:
df_emdat_processing =df_emdat_processing.rename(columns={'myDt':'date', 'Country Code':'country code'})

df_emdat_clean = df_emdat_processing
df_emdat_clean.head()

,Year,Country,country code,Continent,Total Affected,CPI,date
0,1960,Chile,CHL,Americas,2003000.0,11.427251,1960-05-22
1,1960,Algeria,DZA,Africa,1250.0,11.427251,1960-02-21
3,1962,Albania,ALB,Europe,154.0,11.688060,1962-03-18
4,1962,Colombia,COL,Americas,300.0,11.688060,1962-07-30
5,1964,Azores Islands,AZO,Europe,1000.0,11.984281,1964-02-18


In [31]:
df_merged_emdat_usgs= df_emdat_clean.merge(df_usgs_clean, how= 'inner', on=['date', 'country code'])


In [32]:
df_merged_emdat_usgs.head()

,Year,Country,country code,Continent,Total Affected,CPI,date,depth,mag,country,mmi
0,1960,Chile,CHL,Americas,2003000.0,11.427251,1960-05-22,25.0,9.5,Chile,8.842
1,1960,Chile,CHL,Americas,2003000.0,11.427251,1960-05-22,25.0,7.8,Chile,7.274
2,1960,Chile,CHL,Americas,2003000.0,11.427251,1960-05-22,25.0,6.8,Chile,6.925
3,1960,Chile,CHL,Americas,2003000.0,11.427251,1960-05-22,25.0,7.1,Chile,7.112
4,1960,Chile,CHL,Americas,2003000.0,11.427251,1960-05-22,25.0,5.8,Chile,5.841


In [33]:
#drop columns
df_merged_emdat_usgs=df_merged_emdat_usgs.drop(columns=['Country'])

In [34]:
df_merged_emdat_usgs.head()

,Year,country code,Continent,Total Affected,CPI,date,depth,mag,country,mmi
0,1960,CHL,Americas,2003000.0,11.427251,1960-05-22,25.0,9.5,Chile,8.842
1,1960,CHL,Americas,2003000.0,11.427251,1960-05-22,25.0,7.8,Chile,7.274
2,1960,CHL,Americas,2003000.0,11.427251,1960-05-22,25.0,6.8,Chile,6.925
3,1960,CHL,Americas,2003000.0,11.427251,1960-05-22,25.0,7.1,Chile,7.112
4,1960,CHL,Americas,2003000.0,11.427251,1960-05-22,25.0,5.8,Chile,5.841


In [35]:
df_merged_emdat_usgs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1253 entries, 0 to 1252
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year            1253 non-null   int64  
 1   country code    1253 non-null   object 
 2   Continent       1253 non-null   object 
 3   Total Affected  1253 non-null   float64
 4   CPI             1253 non-null   float64
 5   date            1253 non-null   object 
 6   depth           1253 non-null   float64
 7   mag             1253 non-null   float64
 8   country         1253 non-null   object 
 9   mmi             853 non-null    float64
dtypes: float64(5), int64(1), object(4)
memory usage: 107.7+ KB


In [36]:
df_merged_emdat_usgs.to_csv(r'merged_usgs_emdat.csv')

In [41]:
df_merged_emdat_usgs['country'].unique()

array(['Chile', 'Albania', 'Colombia', 'Greece', 'France', 'Australia',
       'Ethiopia', 'Indonesia', 'Costa Rica', 'Afghanistan', 'China',
       'Ecuador', 'Guatemala', 'Argentina', 'Iran', 'Italy', 'Libya',
       'Mexico', 'Turkey', 'Taiwan', 'United Republic of Tanzania',
       'El Salvador', 'Nepal', 'New Zealand', 'Peru',
       'United States of America', 'Nicaragua', 'Pakistan', 'Philippines',
       'Iceland', 'Solomon Islands', 'Romania', 'Japan',
       'Democratic Republic of the Congo', 'South Africa', 'Algeria',
       'Belgium', 'Guinea', 'Bulgaria', 'Brazil', 'Bangladesh',
       'Honduras', 'Papua New Guinea', 'India', 'Venezuela', 'Egypt',
       'Malawi', 'Georgia', 'Myanmar', 'Kyrgyzstan', 'Sudan',
       'Azerbaijan', 'Bolivia', 'Cyprus', 'Russia', 'Slovenia',
       'Tajikistan', 'Dominican Republic', 'Panama', 'Morocco',
       'Mozambique', 'Rwanda', 'Bhutan', 'Haiti', 'Spain',
       'Republic of Serbia', 'Thailand', 'South Korea', 'North Macedonia',
      

# World Bank Indicators

Using the [Word Bank Indicators (WBI) dataset](https://databank.worldbank.org/source/world-development-indicators), we explore different socioeconomic aspects of countries. For the sake of having only a prototype, we explore only a few indicators which can refer to more general aspects of the country infrastructure.

In [42]:
df_wbi = pd.read_csv('WBI-dataset.csv')

In [43]:
country = list(df_wbi['Country Name'].unique())
index = country.index('Zimbabwe')
country = country[:index+1]

In [44]:
codes = list(df_wbi['Country Code'].dropna().unique())
# Remove 'world' variable
codes = codes[:index+1]

In [45]:
def wbi_country(code):
    ''' Reshape world bank dataset input
    code: string code of a country'''
    df_wbi_code = df_wbi[df_wbi['Country Code']== code]
    assert len(df_wbi_code) > 0, 'Does not find ' + code
    country = df_wbi_code.iloc[0]['Country Name']

    # Reshaping dataset 
    df_wbi_code=df_wbi_code.drop(columns=['Country Name', 'Country Code']).T
    df_wbi_code.columns = list(df_wbi_code.iloc[0,:])

    # cleaning rows and adding country columns back
    df_wbi_code = df_wbi_code.drop(['Series Name', 'Series Code'])
    df_wbi_code['country'] = country
    df_wbi_code['country code'] = code

    df_wbi_code.index = df_wbi_code.index.map(lambda x: int(x[0:4]))
    df_wbi_code['Year'] = df_wbi_code.index
    df_wbi_code.reset_index(level=0,drop = True, inplace=True)
    return df_wbi_code


In [46]:
# loop for all countries
dfs = list()

for code in codes:
    dfs.append(wbi_country(code))

wbi_processed = pd.concat(dfs)
wbi_processed.reset_index(inplace=True, drop=True)
wbi_processed.tail()    

,Population density (people per sq. km of land area),Population growth (annual %),Population in largest city,Population in the largest city (% of urban population),"Population, total",Rural population,Rural population (% of total population),Urban population,Urban population (% of total population),Urban population growth (annual %),GDP (constant 2015 US$),GDP (current US$),GDP growth (annual %),GDP per capita (constant 2015 US$),country,country code,Year
13232,36.2681607858343,1.54929408793871,1504803,33.2095334652472,14030338,9499100,67.704,4531238,32.296,1.27409384280022,20114015690.1416,20548678100,0.755869250930601,1433.60877622062,Zimbabwe,ZWE,2016
13233,36.8013416052734,1.45940581314681,1509901,32.8993744786959,14236599,9647147,67.763,4589452,32.237,1.27654356252103,21061283685.9496,17584890936.6523,4.70949217898975,1479.23346812428,Zimbabwe,ZWE,2017
13234,37.3240584205764,1.41038154233327,1515016,32.5768068056639,14438812,9788215,67.791,4650597,32.209,1.32349714884316,22077324353.2946,18115543790.7855,4.82421053956361,1509.52373452279,Zimbabwe,ZWE,2018
13235,37.8582732325191,1.42114188586135,1521311,32.2495652710328,14645473,9928166,67.79,4717307,32.21,1.42424874688139,20720841373.1335,19284289739.0517,-6.14423631439116,1367.67691490163,Zimbabwe,ZWE,2019
13236,38.4203877471888,1.47387141587655,1529920,31.9258446966417,14862927,10070822,67.758,4792105,32.242,1.57316861683581,19426048165.877,18051170798.941,-6.24874822378281,1239.85358358565,Zimbabwe,ZWE,2020


In [47]:
wbi_processed.to_csv(r'wbi_clean.csv')

# Final Merge of all datasets

Now we can finally merge all datasets: `usgs_clean`, `emdat_clean` and `wbi_clean`.

The first two datasets are already merged into the `df_merged_emdat_usgs`, which we now need to merge to the final dataset.

To do so, we will merge the datasets on `Year` and `country code`. However, we first need to check if the list of countries in both datasets are consistent.

In [48]:
# countries that are in the already merged dataset, but not in wbi dataset
diff_country_merged= set(df_merged_emdat_usgs['country']).difference(set(wbi_processed['country']))

# country codes that are in the already merged dataset, but not in wbi dataset
diff_code_merged= set(df_merged_emdat_usgs['country code']).difference(set(wbi_processed['country code']))

print('Countries in merged dataset but not in WBI:', sorted(diff_country_merged))

print('Country codes in merged dataset but not in WBI:', sorted(diff_code_merged))

Countries in merged dataset but not in WBI: ['Democratic Republic of the Congo', 'Egypt', 'Iran', 'Kyrgyzstan', 'Republic of Serbia', 'Russia', 'South Korea', 'Taiwan', 'United Republic of Tanzania', 'United States of America', 'Venezuela']
Country codes in merged dataset but not in WBI: ['TWN']


In [49]:
# Countries code dictionary (WB dataset)

codes= list(wbi_processed['country code'][:])
countries= list(wbi_processed['country'][:])
countries_dict= dict()

for (code,country) in zip(codes,countries):
    countries_dict[code]= country
     
pprint(countries_dict)        

{'ABW': 'Aruba',
 'AFG': 'Afghanistan',
 'AGO': 'Angola',
 'ALB': 'Albania',
 'AND': 'Andorra',
 'ARE': 'United Arab Emirates',
 'ARG': 'Argentina',
 'ARM': 'Armenia',
 'ASM': 'American Samoa',
 'ATG': 'Antigua and Barbuda',
 'AUS': 'Australia',
 'AUT': 'Austria',
 'AZE': 'Azerbaijan',
 'BDI': 'Burundi',
 'BEL': 'Belgium',
 'BEN': 'Benin',
 'BFA': 'Burkina Faso',
 'BGD': 'Bangladesh',
 'BGR': 'Bulgaria',
 'BHR': 'Bahrain',
 'BHS': 'Bahamas, The',
 'BIH': 'Bosnia and Herzegovina',
 'BLR': 'Belarus',
 'BLZ': 'Belize',
 'BMU': 'Bermuda',
 'BOL': 'Bolivia',
 'BRA': 'Brazil',
 'BRB': 'Barbados',
 'BRN': 'Brunei Darussalam',
 'BTN': 'Bhutan',
 'BWA': 'Botswana',
 'CAF': 'Central African Republic',
 'CAN': 'Canada',
 'CHE': 'Switzerland',
 'CHI': 'Channel Islands',
 'CHL': 'Chile',
 'CHN': 'China',
 'CIV': "Cote d'Ivoire",
 'CMR': 'Cameroon',
 'COD': 'Congo, Dem. Rep.',
 'COG': 'Congo, Rep.',
 'COL': 'Colombia',
 'COM': 'Comoros',
 'CPV': 'Cabo Verde',
 'CRI': 'Costa Rica',
 'CUB': 'Cuba',
 '

In [60]:
# Renaming countries in the WBI dataset to match the merged dataset names

wbi_clean = wbi_processed

wbi_clean.loc[wbi_clean.country == 'PRK', 'country code'] = 'KOR'    
wbi_clean.loc[wbi_clean.country == 'COG', 'country code'] = 'COD'    
wbi_clean.loc[wbi_clean.country == 'Congo, Dem. Rep.', 'country'] = 'Democratic Republic of the Congo'    
wbi_clean.loc[wbi_clean.country == 'Congo, Rep.', 'country'] = 'Democratic Republic of the Congo'
wbi_clean.loc[wbi_clean.country == 'Egypt, Arab Rep.', 'country'] = 'Egypt'
wbi_clean.loc[wbi_clean.country == 'Iran, Islamic Rep.', 'country'] = 'Iran'
wbi_clean.loc[wbi_clean.country == 'Kyrgyz Republic', 'country'] = 'Kyrgyzstan'
wbi_clean.loc[wbi_clean.country == 'Serbia', 'country'] = 'Republic of Serbia'
wbi_clean.loc[wbi_clean.country == 'Russian Federation', 'country'] = 'Russia'
wbi_clean.loc[wbi_clean.country == "Korea, Dem. People's Rep.", 'country'] = 'South Korea'
wbi_clean.loc[wbi_clean.country == 'Tanzania', 'country'] = 'United Republic of Tanzania'
wbi_clean.loc[wbi_clean.country == 'Venezuela, RB', 'country'] = 'Venezuela'
wbi_clean.loc[wbi_clean.country == 'United States', 'country'] = 'United States of America'

From that analysis, we see that data for `Taiwan` is missing in the `wbi_clean` dataset. So let's decide how to deal with it by looking into how many datapoints this corresponds to our dataset.

In [61]:
len(df_merged_emdat_usgs[df_merged_emdat_usgs['country code']== 'TWN'])

0

As data for Taiwan corresponds to only 10 datapoints, we will remove those from this analysis. In the future, one can look into adding socioeconomic information about the country to the dataset.

In [62]:
df_merged_emdat_usgs = df_merged_emdat_usgs[df_merged_emdat_usgs['country code'] != 'TWN']

In [63]:
# Drop all countries not relevant from wbi dataset
countries_relevant =  list(df_merged_emdat_usgs['country'].unique())
wbi_clean = wbi_clean[wbi_clean['country'].isin(countries_relevant)]

In [64]:
# final merge 
df_merged_all= df_merged_emdat_usgs.merge(wbi_clean, how= 'inner', on=['country code', 'Year'])

df_merged_all.head()

,Year,country code,Continent,Total Affected,CPI,date,depth,mag,country_x,mmi,...,Rural population,Rural population (% of total population),Urban population,Urban population (% of total population),Urban population growth (annual %),GDP (constant 2015 US$),GDP (current US$),GDP growth (annual %),GDP per capita (constant 2015 US$),country_y
0,1960,CHL,Americas,2003000.0,11.427251,1960-05-22,25.0,9.5,Chile,8.842,...,2615894,32.164,5517094,67.836,..,27084687058.6456,4110000000,..,3330.22587253855,Chile
1,1960,CHL,Americas,2003000.0,11.427251,1960-05-22,25.0,7.8,Chile,7.274,...,2615894,32.164,5517094,67.836,..,27084687058.6456,4110000000,..,3330.22587253855,Chile
2,1960,CHL,Americas,2003000.0,11.427251,1960-05-22,25.0,6.8,Chile,6.925,...,2615894,32.164,5517094,67.836,..,27084687058.6456,4110000000,..,3330.22587253855,Chile
3,1960,CHL,Americas,2003000.0,11.427251,1960-05-22,25.0,7.1,Chile,7.112,...,2615894,32.164,5517094,67.836,..,27084687058.6456,4110000000,..,3330.22587253855,Chile
4,1960,CHL,Americas,2003000.0,11.427251,1960-05-22,25.0,5.8,Chile,5.841,...,2615894,32.164,5517094,67.836,..,27084687058.6456,4110000000,..,3330.22587253855,Chile


In [65]:
# Preparing model dataset
#df_merged_all.drop(columns=['country_x', 'country_x'])
df_merged_all =df_merged_all.drop(columns=['country_x'])
df_merged_all = df_merged_all.rename(columns={'country_y':'country'})

columns = list(df_merged_all.columns)
columns.remove('Total Affected')

df_merged_all = df_merged_all[columns + ['Total Affected']]

df_merged_model = df_merged_all.drop(columns=['Year','country code','Continent','date'])

In [66]:
df_merged_all.head()

,Year,country code,Continent,CPI,date,depth,mag,mmi,Population density (people per sq. km of land area),Population growth (annual %),...,Rural population (% of total population),Urban population,Urban population (% of total population),Urban population growth (annual %),GDP (constant 2015 US$),GDP (current US$),GDP growth (annual %),GDP per capita (constant 2015 US$),country,Total Affected
0,1960,CHL,Americas,11.427251,1960-05-22,25.0,9.5,8.842,..,..,...,32.164,5517094,67.836,..,27084687058.6456,4110000000,..,3330.22587253855,Chile,2003000.0
1,1960,CHL,Americas,11.427251,1960-05-22,25.0,7.8,7.274,..,..,...,32.164,5517094,67.836,..,27084687058.6456,4110000000,..,3330.22587253855,Chile,2003000.0
2,1960,CHL,Americas,11.427251,1960-05-22,25.0,6.8,6.925,..,..,...,32.164,5517094,67.836,..,27084687058.6456,4110000000,..,3330.22587253855,Chile,2003000.0
3,1960,CHL,Americas,11.427251,1960-05-22,25.0,7.1,7.112,..,..,...,32.164,5517094,67.836,..,27084687058.6456,4110000000,..,3330.22587253855,Chile,2003000.0
4,1960,CHL,Americas,11.427251,1960-05-22,25.0,5.8,5.841,..,..,...,32.164,5517094,67.836,..,27084687058.6456,4110000000,..,3330.22587253855,Chile,2003000.0


In [67]:
df_merged_all['country'].unique()

array(['Chile', 'Albania', 'Colombia', 'Greece', 'France', 'Australia',
       'Ethiopia', 'Indonesia', 'Costa Rica', 'Afghanistan', 'China',
       'Ecuador', 'Guatemala', 'Argentina', 'Iran', 'Italy', 'Libya',
       'Mexico', 'Turkey', 'United Republic of Tanzania', 'El Salvador',
       'Nepal', 'New Zealand', 'Peru', 'United States of America',
       'Nicaragua', 'Pakistan', 'Philippines', 'Iceland',
       'Solomon Islands', 'Romania', 'Japan',
       'Democratic Republic of the Congo', 'South Africa', 'Algeria',
       'Belgium', 'Guinea', 'Bulgaria', 'Brazil', 'Bangladesh',
       'Honduras', 'Papua New Guinea', 'India', 'Venezuela', 'Egypt',
       'Malawi', 'Georgia', 'Myanmar', 'Kyrgyzstan', 'Sudan',
       'Azerbaijan', 'Bolivia', 'Cyprus', 'Russia', 'Slovenia',
       'Tajikistan', 'Dominican Republic', 'Panama', 'Morocco',
       'Mozambique', 'Rwanda', 'Bhutan', 'Haiti', 'Spain',
       'Republic of Serbia', 'Thailand', 'North Macedonia', 'Iraq'],
      dtype=object)

In [68]:
df_merged_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1241 entries, 0 to 1240
Data columns (total 20 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   CPI                                                     1241 non-null   float64
 1   depth                                                   1241 non-null   float64
 2   mag                                                     1241 non-null   float64
 3   mmi                                                     845 non-null    float64
 4   Population density (people per sq. km of land area)     1241 non-null   object 
 5   Population growth (annual %)                            1241 non-null   object 
 6   Population in largest city                              1241 non-null   object 
 7   Population in the largest city (% of urban population)  1241 non-null   object 
 8   Population, total                     

In [69]:
df_merged_all.to_csv(r'df_merged_all.csv')
df_merged_model.to_csv(r'df_merged_model.csv')

In [50]:
import pandas_profiling
# Profiling Report

profile = pandas_profiling.ProfileReport(df_merged_all, minimal = True)
profile.to_file(output_file="profiling-min-earthquakes.html")

Summarize dataset:   0%|          | 0/32 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]